# CSE6242 - HW3 - Q1

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

Pyspark Imports

In [18]:
#export
### DO NOT MODIFY THIS CELL ###
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp, ceil, coalesce, avg

Initialize PySpark Context

In [3]:
### DO NOT MODIFY THIS CELL ###
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

Define function for loading data

In [4]:
### DO NOT MODIFY THIS CELL ###
def load_data():
    df = sqlContext.read.option("header",True) \
     .csv("yellow_tripdata_2019-01_short.csv")
    return df

### Q1.1

Perform data casting to clean incoming dataset

In [5]:
#export
def clean_data(df):
    '''
    input: df a dataframe
    output: df a dataframe with the all the original columns
    '''
    
    # START YOUR CODE HERE ---------
    df = df.withColumn('passenger_count', col('passenger_count').cast('int'))
    df = df.withColumn('total_amount', col('total_amount').cast('float'))
    df = df.withColumn('tip_amount', col('tip_amount').cast('float'))
    df = df.withColumn('trip_distance', col('trip_distance').cast('float'))
    df = df.withColumn('fare_amount', col('fare_amount').cast('float'))
    df = df.withColumn('tpep_pickup_datetime', col('tpep_pickup_datetime').cast('timestamp'))
    df = df.withColumn('tpep_dropoff_datetime', col('tpep_dropoff_datetime').cast('timestamp'))
    # END YOUR CODE HERE -----------
    
    return df

### Q1.2

Find rate per person for based on how many passengers travel between pickup and dropoff locations. 

In [6]:
#export
def common_pair(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - PULocationID
            - DOLocationID
            - total_passenger_count
            - per_person_rate
            
    per_person_rate is the total_amount per person for a given pair.
    
    '''
    
    # START YOUR CODE HERE ---------
    df = df.filter(col("PULocationID") != col("DOLocationID"))
    df = df.groupBy("PULocationID", "DOLocationID").agg({
        "passenger_count": "sum",
        "total_amount": "sum"
    })
    df = df.withColumnRenamed("sum(passenger_count)", "total_passenger_count") \
           .withColumnRenamed("sum(total_amount)", "sum_total_amount")
    df = df.withColumn("per_person_rate", col("sum_total_amount") / col("total_passenger_count"))
    df = df.orderBy(col("total_passenger_count").desc(), col("per_person_rate").desc())
    df = df.select("PULocationID", "DOLocationID", "total_passenger_count", "per_person_rate").limit(10)
    # END YOUR CODE HERE -----------
    
    return df

### Q1.3

Find trips which trip distances generate the highest tip percentage.

In [7]:
#export
def distance_with_most_tip(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - trip_distance
            - tip_percent
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    
    # START YOUR CODE HERE ---------
    df = df.filter(col("fare_amount") > 2.00)
    df = df.filter(col("trip_distance") > 0)
    df = df.withColumn("tip_percent_calc", (col("tip_amount") * 100) / col("fare_amount"))
    df = df.withColumn("rounded_distance", ceil(col("trip_distance")))
    df = df.groupBy("rounded_distance").agg({"tip_percent_calc": "avg"})
    df = df.withColumnRenamed("rounded_distance", "trip_distance") \
           .withColumnRenamed("avg(tip_percent_calc)", "tip_percent")
    df = df.select("trip_distance", "tip_percent").orderBy(col("tip_percent").desc()).limit(15)
    # END YOUR CODE HERE -----------
    
    return df

### Q1.4

Determine the average speed at different times of day.

In [16]:
#export
def time_with_most_traffic(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - time_of_day
            - am_avg_speed
            - pm_avg_speed
            
    am_avg_speed and pm_avg_speed are the average trip distance / average trip time calculated for each hour
    
    '''
    
    # START YOUR CODE HERE ---------
    df = df.withColumn(
        "duration_hours",
        (col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long")) / 3600
    )
    
    # Extract hour and compute time_of_day (0-11)
    df = df.withColumn("hour", hour(col("tpep_pickup_datetime"))) \
           .withColumn("time_of_day", col("hour") % 12) \
           .withColumn("period", when(col("hour") < 12, "AM").otherwise("PM"))
    
    # Group by time_of_day and period, compute average distance and duration
    df_grouped = df.groupBy("time_of_day", "period").agg(
        avg("trip_distance").alias("avg_distance"),
        avg("duration_hours").alias("avg_duration")
    )
    
    # Calculate average speed, handling division by zero
    df_grouped = df_grouped.withColumn(
        "avg_speed",
        when(col("avg_duration") > 0, col("avg_distance") / col("avg_duration")).otherwise(None)
    )
    
    # Split into AM and PM speeds
    df_am = df_grouped.filter(col("period") == "AM") \
                      .select("time_of_day", "avg_speed") \
                      .withColumnRenamed("avg_speed", "am_avg_speed")
    
    df_pm = df_grouped.filter(col("period") == "PM") \
                      .select("time_of_day", "avg_speed") \
                      .withColumnRenamed("avg_speed", "pm_avg_speed")
    
    # Join AM and PM speeds on time_of_day
    df_final = df_am.join(df_pm, "time_of_day", "full_outer") \
                    .orderBy("time_of_day") \
                    .select("time_of_day", "am_avg_speed", "pm_avg_speed")

    # END YOUR CODE HERE -----------

    return df_final

## The below cells are for you to investigate your solutions and will not be graded

In [10]:
df = load_data()
df = clean_data(df)

In [11]:
common_pair(df).show()

+------------+------------+---------------------+------------------+
|PULocationID|DOLocationID|total_passenger_count|   per_person_rate|
+------------+------------+---------------------+------------------+
|         239|         238|                   62|  4.26274198870505|
|         237|         236|                   60| 4.482500068346659|
|         263|         141|                   52|3.4190384974846473|
|         161|         236|                   42| 5.368571440378825|
|         148|          79|                   42| 4.711904752822149|
|         142|         238|                   39|  5.05487182812813|
|         141|         236|                   37| 4.355675723101641|
|         239|         143|                   37| 4.252162224537617|
|         239|         142|                   35| 3.817714350564139|
|          79|         170|                   34| 6.394705884596881|
+------------+------------+---------------------+------------------+



In [12]:
distance_with_most_tip(df).show()

+-------------+------------------+
|trip_distance|       tip_percent|
+-------------+------------------+
|            1|17.129815971513313|
|            2|15.815527155632552|
|           17|15.796441782308916|
|           20| 15.11240992123345|
|            3|14.886705727113446|
|            6|14.579695131601051|
|            5|14.245405861990653|
|            4|13.831569507473274|
|            9|13.814476557648435|
|            8|12.072596772433315|
|           19|11.952632334985276|
|           10|11.880490518902954|
|            7| 10.80057562837643|
|           21|10.739019886973427|
|           18|10.696822158448429|
+-------------+------------------+



In [19]:
time_with_most_traffic(df).show()

+-----------+------------------+-------------------+
|time_of_day|      am_avg_speed|       pm_avg_speed|
+-----------+------------------+-------------------+
|          0| 9.377696196631234|               null|
|          1|10.845483413697353|  5.125214305177561|
|          3|              null|                0.0|
|          4|              null|                0.0|
|          5|              null| 0.5137660239764732|
|          6|              null|  9.989847870647605|
|          7|              null|0.18415305490417713|
|          8|              null| 0.5183127622697896|
|         10|              null| 0.6147483972627696|
|         11|              null|  4.650958285207579|
+-----------+------------------+-------------------+

